<a href="https://colab.research.google.com/github/Eiko58/ML-in-Finance/blob/main/Ex7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.datasets as datasets
from torch import Tensor
import torch.nn.init as init
import numpy as np
from numpy import save
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
import torch.nn.functional as F
import scipy.stats as scipy
from scipy.stats import norm
import yfinance as yf # https://github.com/ranaroussi/yfinance
import pandas as pd
def BS(p,k,t,sigma):

  sigma = sigma.squeeze(1)
  d1 = (torch.log(p/k)+(0.5*sigma**2)*t)/(sigma*torch.sqrt(t))
  d2 = d1 - sigma*torch.sqrt(t)
  N = torch.distributions.Normal(0,1).cdf
  #print(p*N(dp) - k*N(dm))
  return p*N(d1) - k*N(d2)


In [29]:
aapl= yf.Ticker("AAPL")
exp = aapl.options # list of dates
opt_call = pd.DataFrame()
for e in exp:
  opt = aapl.option_chain(e)
  opt_call = pd.concat([opt_call, opt.calls], ignore_index=True)
  opt_call['expirationDate'] = e
sp = aapl.history(period='1d')['Close'][0]
data = opt_call[['ask','bid','strike','impliedVolatility']]
time_diff = (pd.to_datetime(opt_call.expirationDate).dt.date - opt_call.lastTradeDate.dt.date)
td = pd.to_timedelta(time_diff)
data['time'] = td.dt.days
data['call'] = data[['ask', 'bid']].mean(axis=1)
data['price'] = sp

<ipython-input-29-fe8b9d267c0f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time'] = td.dt.days
<ipython-input-29-fe8b9d267c0f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['call'] = data[['ask', 'bid']].mean(axis=1)
<ipython-input-29-fe8b9d267c0f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [30]:
which = data['call'] > 0
data1 = data.loc[which]
data1['price_norm'] = 1
data1['strike_norm'] = (data1['strike'] - data1['strike'].mean())/data1['strike'].std()
data1['time_norm'] = (data1['time'] - data1['time'].mean())/data1['time'].std()
train = data1.sample(frac=0.8,random_state=200)
train_C = train['call']
train_p = train['price']
train_K = train['strike']
train_t = train['time']
train_pn = train['price_norm']
train_kn = train['strike_norm']
train_tn = train['time_norm']
#train_iv = train['iv']

#train_p = train[['lastPrice','strike','time']]
test = data1.drop(train.index)
test_C = test['call']
test_p = test['price']
test_K = test['strike']
test_t = test['time']
test_pn = test['price_norm']
test_kn = test['strike_norm']
test_tn = test['time_norm']
#test_p = test[['lastPrice','strike','time']]
#test_iv = test['impliedVolatility']

<ipython-input-30-6b032c78fd6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['price_norm'] = 1
<ipython-input-30-6b032c78fd6c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['strike_norm'] = (data1['strike'] - data1['strike'].mean())/data1['strike'].std()
<ipython-input-30-6b032c78fd6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [31]:
test_C

144      0.015
220    139.475
222    143.050
224    109.950
276    161.475
387    102.950
389     88.425
390     76.850
541     92.200
545     77.850
748    136.725
Name: call, dtype: float64

In [32]:
data1

,ask,bid,strike,impliedVolatility,time,call,price,price_norm,strike_norm,time_norm
91,120.35,119.60,50.0,2.849612,1019,119.975,169.300003,1,-0.473839,0.144539
92,115.40,114.65,55.0,2.671878,1019,115.025,169.300003,1,-0.416978,0.144539
142,0.02,0.00,270.0,0.671878,996,0.010,169.300003,1,2.028029,-0.879735
143,0.02,0.00,275.0,0.687503,1004,0.010,169.300003,1,2.084890,-0.523465
144,0.03,0.00,280.0,0.742190,1022,0.015,169.300003,1,2.141751,0.278140
145,0.03,0.00,285.0,0.765627,1015,0.015,169.300003,1,2.198611,-0.033596
147,0.01,0.00,295.0,0.750003,1012,0.005,169.300003,1,2.312333,-0.167196
215,160.80,159.20,5.0,0.000010,1002,160.000,169.300003,1,-0.985584,-0.612533
216,163.75,162.80,10.0,6.475588,1026,163.275,169.300003,1,-0.928724,0.456274
217,155.00,153.25,15.0,3.402345,995,154.125,169.300003,1,-0.871863,-0.924268


In [33]:
class NN(nn.Module):
  def __init__(self,hidden_size):
        super(NN, self).__init__()
        self.linear1 = nn.Linear(3, hidden_size)
        self.bn1 = nn.InstanceNorm1d(hidden_size)
        self.linear2 = nn.Linear(hidden_size, int(hidden_size/2))
        self.bn2 = nn.InstanceNorm1d(int(hidden_size/2))
        self.linear3 = nn.Linear(int(hidden_size/2), 1)
        #self.linear4 = nn.Linear(int(hidden_size/2), 10)
        #self.linear5 = nn.Linear(10, 1)
        init.kaiming_uniform_(self.linear1.weight)
        init.kaiming_uniform_(self.linear2.weight)
        init.kaiming_uniform_(self.linear3.weight,nonlinearity='sigmoid')

  def forward(self, p,k,t):
    p = p.view(len(p), -1)
    k = k.view(len(p), -1)
    t = t.view(len(p), -1)
    x = torch.cat((p,k,t),-1)
    x = self.linear1(x)
    x = self.bn1(x)
    x = F.elu(x)
    x = self.linear2(x)
    x = self.bn2(x)
    x = F.elu(x)
    x = self.linear3(x)

    #x = F.elu(x)
    #out = F.sigmoid(x)
    #x = self.linear4(x)
    #x = F.relu(x)
    #x = self.linear5(x)
    out = F.sigmoid(x)
    return out

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, train, out, C):
        return torch.mean((torch.tensor(BS(train,out))-C)**2)


In [39]:
def train_func(batch_size,nepochs,lr):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # Normalize the tensors before creating the TensorDataset
    #train_C_norm = nn.functional.normalize(Tensor(train_C.values), dim=0)
    #train_p_norm = nn.functional.normalize(Tensor(train_p.values), dim=0)
    #train_K_norm = nn.functional.normalize(Tensor(train_K.values), dim=0)
    #train_t_norm = nn.functional.normalize(Tensor(train_t.values), dim=0)

    # Create the TensorDataset with the normalized tensors
    #data_train = TensorDataset(train_C_norm, train_p_norm, train_K_norm, train_t_norm)
    data_train = TensorDataset(Tensor(train_C.values), Tensor(train_p.values), Tensor(train_K.values), Tensor(train_t.values),
                               Tensor(train_pn.values), Tensor(train_kn.values), Tensor(train_tn.values))
    # Do the same for the test data
    #test_p_norm = nn.functional.normalize(Tensor(test_p.values), dim=0)
    #test_K_norm = nn.functional.normalize(Tensor(test_K.values), dim=0)
    #test_t_norm = nn.functional.normalize(Tensor(test_t.values), dim=0)

    #data_test = TensorDataset(test_p_norm, test_K_norm, test_t_norm, Tensor(test_iv.values))
    data_test = TensorDataset(Tensor(test_C.values), Tensor(test_p.values), Tensor(test_K.values), Tensor(test_t.values),
                              Tensor(test_pn.values), Tensor(test_kn.values), Tensor(test_tn.values))
    train_loader = DataLoader(data_train, batch_size = batch_size,shuffle=False)
    test_loader = DataLoader(data_test,shuffle=False)
    total_step = len(train_loader)
    test_size = len(test_loader)
    model = NN(128).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    #loss_f = CustomLoss()
    for epoch in range(nepochs):
        for batch_idx,(call_price,p,k,t,p_n,k_n,t_n) in enumerate(train_loader):
        #optimizer.zero_grad()
            model.train()
            optimizer.zero_grad()
            call_price = call_price.to(device)
            p = p.to(device)
            k = k.to(device)
            t = t.to(device)
            p_n = p_n.to(device)
            k_n = k_n.to(device)
            t_n = t_n.to(device)
            #parameter = parameter.to(device)
            out_iv = model(p_n,k_n,t_n)
            #loss = torch.mean(((BS(p,k,t,out_iv).clone().detach())-call_price)**2).requires_grad_(True)
            loss = F.mse_loss(BS(p,k,t,out_iv),call_price).requires_grad_(True)
            #loss = F.mse_loss(torch.log(BS(p,k,t,out_iv)),torch.log(call_price)).requires_grad_(True)
            #loss = F.mse_loss(out_iv,call_price)
            #loss_f = CustomLoss()
            #loss =((call_price-out_iv)**2).mean()
            #loss = loss_f(parameter,out_iv,call_price).requires_grad_(True)
            #loss = loss_f(parameter,out_iv,call_price).requires_grad_(True)
            #loss = torch.mean((torch.tensor(BS(train,out_iv))-call_price)**2)
            loss.backward()
            #print (model.linear1.weight.grad)
            #print (model.linear3.weight.grad)
            #print (model.linear5.weight.grad)
            optimizer.step()


            if (batch_idx+1) % 1 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                        .format(epoch+1, nepochs, batch_idx+1, total_step, loss.item()))


        with torch.no_grad():
            model.eval()
            distance_sum = 0
            for C,p,k,t,pn,kn,tn in test_loader:
                #parameter = parameter.to(device)
                call_price = C.to(device)
                p = p.to(device)
                k = k.to(device)
                t = t.to(device)
                pn = pn.to(device)
                kn = kn.to(device)
                tn = tn.to(device)
                out_iv = model(pn,kn,tn)
                loss = F.mse_loss(torch.log(BS(p,k,t,out_iv)),torch.log(call_price))
                distance_sum = distance_sum + loss
            #print('The real implied volatility is {}'.format(iv))
            #print('The predicted implied volatility is {}'.format(out_iv))
            print('Average L2 distance to real call price is {}'.format(distance_sum/test_size))


In [47]:
train_func(5,600,1e-4)

Streaming output truncated to the last 5000 lines.
Epoch [146/600], Step [6/10], Loss: 4.2723
Epoch [146/600], Step [7/10], Loss: 46.7299
Epoch [146/600], Step [8/10], Loss: 30.1328
Epoch [146/600], Step [9/10], Loss: 6.4386
Epoch [146/600], Step [10/10], Loss: 0.3219
Average L2 distance to real implied volatility is 1.2734335660934448
Epoch [147/600], Step [1/10], Loss: 6.6287
Epoch [147/600], Step [2/10], Loss: 10.9727
Epoch [147/600], Step [3/10], Loss: 4.7883
Epoch [147/600], Step [4/10], Loss: 20.9890
Epoch [147/600], Step [5/10], Loss: 5.1384
Epoch [147/600], Step [6/10], Loss: 4.2615
Epoch [147/600], Step [7/10], Loss: 46.6852
Epoch [147/600], Step [8/10], Loss: 30.1213
Epoch [147/600], Step [9/10], Loss: 6.4337
Epoch [147/600], Step [10/10], Loss: 0.3144
Average L2 distance to real implied volatility is 1.2683042287826538
Epoch [148/600], Step [1/10], Loss: 6.6345
Epoch [148/600], Step [2/10], Loss: 10.9613
Epoch [148/600], Step [3/10], Loss: 4.7826
Epoch [148/600], Step [4/10]